# **安装依赖库**

## **作者简介**

#### 作者：**⚡小闪电⚡**

#### B站主页
- [小闪电的B站主页](https://space.bilibili.com/122699831?spm_id_from=333.1007.0.0)

#### 交流群
欢迎加入AIDD交流群！  
加我微信（微信号: `xxxFLASHxxx`），邀请你进群。

## **安装依赖库**

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install rdkit
!pip install networkx pandas
%matplotlib inline
import matplotlib.pyplot as plt
from torch_geometric.datasets import KarateClub
dataset = KarateClub()
print(f'Dataset: {dataset}:')

In [ ]:
!pip install tabulate

# **第1部分：一步一步来，先试试简单的二模态融合**

## **Graph+描述符/指纹**

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy.sparse import coo_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

# RDKit & Sklearn
from rdkit import Chem
from rdkit.Chem import rdchem, rdBase, Descriptors, AllChem
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler

# PyG (自动检测安装)
try:
    from torch_geometric.data import Data, Batch
    from torch_geometric.nn import GATConv, global_mean_pool
except ImportError:
    print("正在安装 PyG 依赖...")
    os.system("pip install -q torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cu118.html")
    os.system("pip install -q torch-geometric")
    from torch_geometric.data import Data, Batch
    from torch_geometric.nn import GATConv, global_mean_pool

# 配置环境
import warnings
warnings.filterwarnings('ignore')
rdBase.DisableLog('rdApp.warning')

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(42)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {DEVICE}")

# 绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 150

# ==============================================================================
# 1. 严格复刻原 GNN 特征提取代码 (Feature Extraction)
# ==============================================================================

# --- 辅助函数 ---
def one_hot_encoding(value, choices):
    encoding = [0] * (len(choices) + 1)
    index = choices.index(value) if value in choices else -1
    encoding[index] = 1
    return encoding

# --- 原版 MoleculeFeaturizer (保持不变) ---
class MoleculeFeaturizer(object):
    def _atom_featurizer(self, atom):
        # 37种原子 + 1种其他
        atomic_numer = [1, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 46, 47, 48, 49, 50, 51, 52, 53]
        return (one_hot_encoding(atom.GetAtomicNum(), atomic_numer) +
                one_hot_encoding(atom.GetTotalDegree(), list(range(5))) +
                one_hot_encoding(int(atom.GetHybridization()), list(range(len(Chem.HybridizationType.names)-1))) +
                one_hot_encoding(atom.GetChiralTag(), list(range(len(Chem.ChiralType.names)-1))) +
                one_hot_encoding(atom.GetTotalNumHs(), list(range(5))) +
                [1 if atom.GetIsAromatic() else 0])

    def _bond_featurizer(self, bond):
        bt = [
            int(bond.GetBondType() == rdchem.BondType.SINGLE),
            int(bond.GetBondType() == rdchem.BondType.DOUBLE),
            int(bond.GetBondType() == rdchem.BondType.TRIPLE),
            int(bond.GetBondType() == rdchem.BondType.AROMATIC)
        ]
        return bt + [int(bond.GetIsConjugated()), int(bond.IsInRing())]

    def __call__(self, mol):
        atom_features = [self._atom_featurizer(atom) for atom in mol.GetAtoms()]
        x = torch.tensor(atom_features, dtype=torch.float32)

        adj = Chem.GetAdjacencyMatrix(mol)
        coo_adj = coo_matrix(adj)
        row, col = coo_adj.row, coo_adj.col
        edge_index = torch.tensor([row, col], dtype=torch.long)

        bond_features = []
        for i, j in zip(row, col):
            bond = mol.GetBondBetweenAtoms(int(i), int(j))
            bond_features.append(self._bond_featurizer(bond))

        edge_attr = torch.tensor(bond_features, dtype=torch.float32) if len(bond_features) > 0 else torch.empty((0, 6), dtype=torch.float32)
        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

# --- 辅助数据提取函数 ---
def get_descriptors(mol):
    # 常用 9 种描述符
    return [
        Descriptors.MolWt(mol), Descriptors.MolLogP(mol),
        Descriptors.NumHDonors(mol), Descriptors.NumHAcceptors(mol),
        Descriptors.TPSA(mol), Descriptors.NumRotatableBonds(mol),
        Descriptors.FractionCSP3(mol), Descriptors.HallKierAlpha(mol),
        Descriptors.RingCount(mol)
    ]

def get_fingerprint(mol, n_bits=1024):
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=n_bits)
    return np.array(fp)

# ==============================================================================
# 2. 数据集定义
# ==============================================================================
class FusionDataset(Dataset):
    def __init__(self, df, featurizer):
        self.data = []
        print("正在构建多模态数据集...")

        # 收集描述符以进行归一化
        all_descs = []
        temp_data = []

        for _, row in tqdm(df.iterrows(), total=len(df)):
            mol = Chem.MolFromSmiles(row['Smiles'])
            if mol is None: continue

            # 1. Graph Data (Data对象)
            graph = featurizer(mol)

            # 2. Descriptors (List)
            desc = get_descriptors(mol)

            # 3. Fingerprints (Array)
            fp = get_fingerprint(mol, n_bits=1024)

            all_descs.append(desc)
            temp_data.append({
                'graph': graph,
                'fp': torch.tensor(fp, dtype=torch.float32),
                'y': torch.tensor([row['logSw']], dtype=torch.float32)
            })

        # 描述符标准化 (Z-Score)
        scaler = StandardScaler()
        all_descs_scaled = scaler.fit_transform(all_descs)

        for i, item in enumerate(temp_data):
            item['desc'] = torch.tensor(all_descs_scaled[i], dtype=torch.float32)
            self.data.append(item)

    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]

def collate_fn(batch):
    graphs = Batch.from_data_list([x['graph'] for x in batch])
    descs = torch.stack([x['desc'] for x in batch])
    fps = torch.stack([x['fp'] for x in batch])
    labels = torch.cat([x['y'] for x in batch])
    return graphs, descs, fps, labels

# ==============================================================================
# 3. 模型定义 (严格复刻 + 融合)
# ==============================================================================

# --- 3.1 权重初始化 (原代码中的函数) ---
def init_weight(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

# --- 3.2 GATNet + MLP (结构完全一致，只是 Forward 稍微改动以输出特征) ---
class GAT_FeatureExtractor(nn.Module):
    def __init__(self, input_dim=69, hidden_dim=16, mlp_hidden=128, heads=2, edge_dim=6, dropout=0.4):
        super(GAT_FeatureExtractor, self).__init__()

        # === 严格复刻 GATNet 结构 ===
        self.gat1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=dropout, edge_dim=edge_dim)
        self.gat2 = GATConv(hidden_dim * heads, hidden_dim, heads=1, dropout=dropout, edge_dim=edge_dim)
        self.pool = global_mean_pool

        # === 严格复刻 MLP 结构 (除了最后一层) ===
        # 原 MLP: Linear(16->128) -> ReLU -> Drop(0.4) -> Linear(128->64) -> ReLU -> Drop(0.4) -> Linear(64->1)
        # 这里我们只取前两层作为特征提取器，输出 64 维向量
        self.mlp_part1 = nn.Sequential(
            nn.Linear(hidden_dim, mlp_hidden), # 16 -> 128
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_hidden, mlp_hidden // 2), # 128 -> 64
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        # 初始化权重
        self.mlp_part1.apply(init_weight)

    def forward(self, x, edge_index, batch, edge_attr):
        # GAT Forward
        x = self.gat1(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)
        x = self.gat2(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)
        x = self.pool(x, batch) # [Batch, 16]

        # MLP Forward (Part 1)
        feat = self.mlp_part1(x) # [Batch, 64]
        return feat

# --- 3.3 融合模型 (适配三种模式) ---
class FusionModel(nn.Module):
    def __init__(self, mode, aux_dim=0):
        """
        mode: 'only' (仅图), 'desc' (图+描述符), 'fp' (图+指纹)
        aux_dim: 辅助特征维度 (仅当 mode != 'only' 时有用)
        """
        super(FusionModel, self).__init__()
        self.mode = mode

        # 1. Graph Branch (参数严格锁定为原代码默认值)
        # input_dim=69, hidden=16, mlp_hidden=128, heads=2, edge=6, drop=0.4
        self.graph_encoder = GAT_FeatureExtractor()

        # 2. Auxiliary Branch & Fusion Head
        graph_feat_dim = 64

        if mode == 'only':
            # GNN Only: 直接接原代码的最后一层 MLP
            self.head = nn.Linear(graph_feat_dim, 1) # Linear(64->1)
            self.aux_encoder = None

        else:
            # GNN + Aux
            self.aux_encoder = nn.Sequential(
                nn.Linear(aux_dim, 128),
                nn.ReLU(),
                nn.Dropout(0.4), # 保持一致的 dropout
                nn.Linear(128, 64),
                nn.ReLU()
            )

            # Fusion Head
            # 输入: Graph(64) + Aux(64) = 128
            self.head = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(64, 1)
            )

        self.apply(init_weight)

    def forward(self, graph, aux_data=None):
        # Graph Feature
        g_feat = self.graph_encoder(graph.x, graph.edge_index, graph.batch, graph.edge_attr)

        if self.mode == 'only':
            return self.head(g_feat).squeeze()

        # Aux Feature
        a_feat = self.aux_encoder(aux_data)

        # Concat
        combined = torch.cat([g_feat, a_feat], dim=1)

        # Predict
        return self.head(combined).squeeze()

# ==============================================================================
# 4. 训练与评估脚本
# ==============================================================================
def train_step(model, loader, optimizer, criterion, mode):
    model.train()
    total_loss = 0
    for graphs, descs, fps, labels in loader:
        graphs = graphs.to(DEVICE)

        if mode == 'desc': aux = descs.to(DEVICE)
        elif mode == 'fp': aux = fps.to(DEVICE)
        else: aux = None

        labels = labels.to(DEVICE)

        optimizer.zero_grad()
        preds = model(graphs, aux)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * labels.size(0)
    return total_loss / len(loader.dataset)

def eval_step(model, loader, mode):
    model.eval()
    preds_all, labels_all = [], []
    with torch.no_grad():
        for graphs, descs, fps, labels in loader:
            graphs = graphs.to(DEVICE)

            if mode == 'desc': aux = descs.to(DEVICE)
            elif mode == 'fp': aux = fps.to(DEVICE)
            else: aux = None

            preds = model(graphs, aux)
            preds_all.extend(preds.cpu().numpy())
            labels_all.extend(labels.numpy())

    mse = mean_squared_error(labels_all, preds_all)
    r2 = r2_score(labels_all, preds_all)
    return mse, r2, np.array(labels_all), np.array(preds_all)

# ==============================================================================
# 5. 主程序
# ==============================================================================
def main():
    # 1. 准备数据
    csv_path = '/content/SweetpredDB.csv'
    if not os.path.exists(csv_path):
        print("Creating dummy data...")
        df = pd.DataFrame({'Smiles': ['C']*20 + ['CC']*20, 'logSw': np.random.rand(40)})
    else:
        df = pd.read_csv(csv_path)
        df.rename(columns=str.lower, inplace=True)
        df.rename(columns={'smiles': 'Smiles', 'logsw': 'logSw'}, inplace=True)

    featurizer = MoleculeFeaturizer()
    dataset = FusionDataset(df, featurizer)

    # 划分 (8:2) - 也可以改成 8:1:1，但为了简洁对比这里用 8:2
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_ds, test_ds = random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))

    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_ds, batch_size=32, collate_fn=collate_fn)

    # 获取维度
    sample = dataset[0]
    desc_dim = sample['desc'].shape[0] # 9
    fp_dim = sample['fp'].shape[0]     # 1024

    print(f"\nDimensions -> Descriptors: {desc_dim}, Fingerprints: {fp_dim}")

    # 2. 初始化三个模型
    # Model A: GNN Only
    model_only = FusionModel('only').to(DEVICE)
    # Model B: Graph + Descriptors
    model_desc = FusionModel('desc', desc_dim).to(DEVICE)
    # Model C: Graph + Fingerprints
    model_fp = FusionModel('fp', fp_dim).to(DEVICE)

    # 3. 优化器 (严格复刻原代码参数: lr=0.001, weight_decay=2e-3)
    opt_only = optim.Adam(model_only.parameters(), lr=0.001, weight_decay=2e-3)
    opt_desc = optim.Adam(model_desc.parameters(), lr=0.001, weight_decay=2e-3)
    opt_fp = optim.Adam(model_fp.parameters(), lr=0.001, weight_decay=2e-3)
    criterion = nn.MSELoss()

    # 4. 训练循环
    epochs = 200
    history = {'only_loss': [], 'desc_loss': [], 'fp_loss': [],
               'only_r2': [], 'desc_r2': [], 'fp_r2': []}

    print("\n🚀 开始三模型对比训练...")
    for epoch in range(epochs):
        # Train
        loss_o = train_step(model_only, train_loader, opt_only, criterion, 'only')
        loss_d = train_step(model_desc, train_loader, opt_desc, criterion, 'desc')
        loss_f = train_step(model_fp, train_loader, opt_fp, criterion, 'fp')

        # Eval (Test Set Monitoring)
        _, r2_o, _, _ = eval_step(model_only, test_loader, 'only')
        _, r2_d, _, _ = eval_step(model_desc, test_loader, 'desc')
        _, r2_f, _, _ = eval_step(model_fp, test_loader, 'fp')

        history['only_loss'].append(loss_o)
        history['desc_loss'].append(loss_d)
        history['fp_loss'].append(loss_f)

        history['only_r2'].append(r2_o)
        history['desc_r2'].append(r2_d)
        history['fp_r2'].append(r2_f)

        if (epoch+1) % 1 == 0:
            print(f"Ep {epoch+1:03d} | [Only] R2:{r2_o:.3f} | [Desc] R2:{r2_d:.3f} | [FP] R2:{r2_f:.3f}")

    # 5. 最终评估与可视化
    print("\n✅ 训练完成，生成对比图表...")
    _, final_r2_o, y_true, y_pred_o = eval_step(model_only, test_loader, 'only')
    _, final_r2_d, _, y_pred_d = eval_step(model_desc, test_loader, 'desc')
    _, final_r2_f, _, y_pred_f = eval_step(model_fp, test_loader, 'fp')

    # === 绘图 ===
    fig = plt.figure(figsize=(18, 6))

    # 子图1：Loss 曲线
    ax1 = plt.subplot(1, 3, 1)
    ax1.plot(history['only_loss'], label='GNN Only', color='gray', linestyle='--')
    ax1.plot(history['desc_loss'], label='GNN + Descriptors', color='tab:blue')
    ax1.plot(history['fp_loss'], label='GNN + Fingerprints', color='tab:red')
    ax1.set_title('Training Loss')
    ax1.set_xlabel('Epoch')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # 子图2：R2 曲线
    ax2 = plt.subplot(1, 3, 2)
    ax2.plot(history['only_r2'], label='GNN Only', color='gray', linestyle='--')
    ax2.plot(history['desc_r2'], label='GNN + Descriptors', color='tab:blue')
    ax2.plot(history['fp_r2'], label='GNN + Fingerprints', color='tab:red')
    ax2.set_title('Test R2 Score')
    ax2.set_xlabel('Epoch')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # 子图3：最终预测散点对比
    ax3 = plt.subplot(1, 3, 3)
    # 绘制 GNN Only (底色)
    ax3.scatter(y_true, y_pred_o, alpha=0.3, color='gray', label=f'Only (R2={final_r2_o:.3f})', s=10)
    # 绘制 GNN+Desc
    ax3.scatter(y_true, y_pred_d, alpha=0.5, color='tab:blue', label=f'+Desc (R2={final_r2_d:.3f})', s=20, marker='^')
    # 绘制 GNN+FP
    ax3.scatter(y_true, y_pred_f, alpha=0.5, color='tab:red', label=f'+FP (R2={final_r2_f:.3f})', s=20, marker='x')

    min_v, max_v = y_true.min(), y_true.max()
    ax3.plot([min_v, max_v], [min_v, max_v], 'k--', lw=1.5)
    ax3.set_title('Prediction Correlation')
    ax3.set_xlabel('True Sweetness')
    ax3.set_ylabel('Predicted')
    ax3.legend()
    ax3.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\n🏆 最终 R2 对比:")
    print(f"   - GNN Only: {final_r2_o:.4f}")
    print(f"   - GNN + Descriptors: {final_r2_d:.4f}")
    print(f"   - GNN + Fingerprints: {final_r2_f:.4f}")

if __name__ == '__main__':
    main()

## **SMIELS+描述符/指纹**

In [ ]:
import os
import re
import math
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence

# RDKit & Sklearn
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, rdBase
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

# ==============================================================================
# 0. Global Configuration
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')
rdBase.DisableLog('rdApp.warning')

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(42)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {DEVICE}")

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 150

# ==============================================================================
# 1. Core Components: Tokenizer & Featurizers
# ==============================================================================

# --- SMILES Tokenizer ---
class SMILESTokenizer:
    def __init__(self, max_len=128):
        self.max_len = max_len
        self.pattern =  r"(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"
        self.regex = re.compile(self.pattern)
        self.special_tokens = ["<pad>", "<unk>", "<sos>", "<eos>"]
        self.vocab = {t: i for i, t in enumerate(self.special_tokens)}
        self.unk_token = "<unk>"
        self.sos_token = "<sos>"
        self.eos_token = "<eos>"
        self.pad_token = "<pad>"

    def tokenize(self, smiles):
        return [token for token in self.regex.findall(smiles)]

    def build_vocab(self, smiles_list):
        counter = Counter()
        for s in smiles_list:
            counter.update(self.tokenize(s))
        for token, _ in counter.most_common():
            if token not in self.vocab:
                self.vocab[token] = len(self.vocab)
        return self.vocab

    def encode(self, smiles):
        tokens = self.tokenize(smiles)[:self.max_len - 2]
        ids = [self.vocab.get(t, self.vocab[self.unk_token]) for t in tokens]
        return [self.vocab[self.sos_token]] + ids + [self.vocab[self.eos_token]]

# --- Auxiliary Feature Extractors ---
def get_descriptors(mol):
    # 9 common physicochemical properties
    return [
        Descriptors.MolWt(mol), Descriptors.MolLogP(mol),
        Descriptors.NumHDonors(mol), Descriptors.NumHAcceptors(mol),
        Descriptors.TPSA(mol), Descriptors.NumRotatableBonds(mol),
        Descriptors.FractionCSP3(mol), Descriptors.HallKierAlpha(mol),
        Descriptors.RingCount(mol)
    ]

def get_fingerprint(mol, n_bits=1024):
    # Morgan Fingerprint (ECFP4)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=n_bits)
    return np.array(fp)

# ==============================================================================
# 2. Dataset Construction
# ==============================================================================
class TransFusionDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.data = []
        print("Preprocessing data (SMILES Encoding + Descriptors + Fingerprints)...")

        all_descs = []
        temp_data = []

        for _, row in tqdm(df.iterrows(), total=len(df)):
            mol = Chem.MolFromSmiles(row['Smiles'])
            if mol is None: continue

            # 1. Sequence (Transformer Input)
            seq_ids = tokenizer.encode(row['Smiles'])

            # 2. Descriptors (Aux A)
            desc = get_descriptors(mol)

            # 3. Fingerprints (Aux B)
            fp = get_fingerprint(mol)

            all_descs.append(desc)
            temp_data.append({
                'seq': torch.tensor(seq_ids, dtype=torch.long),
                'fp': torch.tensor(fp, dtype=torch.float32),
                'y': torch.tensor([row['logSw']], dtype=torch.float32)
            })

        # Standardize descriptors
        scaler = StandardScaler()
        all_descs_scaled = scaler.fit_transform(all_descs)

        for i, item in enumerate(temp_data):
            item['desc'] = torch.tensor(all_descs_scaled[i], dtype=torch.float32)
            self.data.append(item)

    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]

def collate_fn(batch):
    # Pad Sequence for Transformer
    pad_id = 0
    seqs = pad_sequence([x['seq'] for x in batch], batch_first=True, padding_value=pad_id)

    descs = torch.stack([x['desc'] for x in batch])
    fps = torch.stack([x['fp'] for x in batch])
    labels = torch.cat([x['y'] for x in batch])

    return seqs, descs, fps, labels

# ==============================================================================
# 3. Model Architecture (Transformer Encoder + Fusion)
# ==============================================================================

# --- 3.1 Transformer Branch ---
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=4, num_layers=3, dim_feedforward=256, pad_idx=0, dropout=0.1):
        super().__init__()
        self.pad_idx = pad_idx
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_encoder = PositionalEncoding(d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead,
            dim_feedforward=dim_feedforward,
            batch_first=True, dropout=dropout
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_dim = d_model

    def forward(self, x):
        # x: [Batch, Seq]
        padding_mask = (x == self.pad_idx)

        x = self.embedding(x) * math.sqrt(self.output_dim)
        x = self.pos_encoder(x)
        x = self.transformer(x, src_key_padding_mask=padding_mask)

        # Mean Pooling (Masked)
        mask_expanded = (~padding_mask).unsqueeze(-1).float()
        sum_embeddings = torch.sum(x * mask_expanded, dim=1)
        sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
        feature = sum_embeddings / sum_mask
        return feature

# --- 3.2 Fusion Architecture ---
class TransFusionModel(nn.Module):
    def __init__(self, mode, vocab_size, aux_dim=0, pad_idx=0):
        """
        mode: 'only' (SMILES Only), 'desc' (SMILES + Desc), 'fp' (SMILES + FP)
        """
        super().__init__()
        self.mode = mode

        # 1. Transformer Branch (Fixed Parameters)
        self.trans = TransformerEncoder(
            vocab_size=vocab_size,
            d_model=128, nhead=4, num_layers=3,
            dim_feedforward=256, pad_idx=pad_idx, dropout=0.1
        )

        trans_out_dim = 128

        if mode == 'only':
            # Single Modality
            self.aux_mlp = None
            self.head = nn.Sequential(
                nn.Linear(trans_out_dim, 64),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(64, 1)
            )
        else:
            # Dual Modality
            # Map auxiliary features to 128 dim
            self.aux_mlp = nn.Sequential(
                nn.Linear(aux_dim, 128),
                nn.BatchNorm1d(128),
                nn.ReLU(),
                nn.Dropout(0.2)
            )

            # Fusion Head (Concat: 128 + 128 -> 256)
            self.head = nn.Sequential(
                nn.Linear(256, 64),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(64, 1)
            )

    def forward(self, seq, aux_data=None):
        # Path A: Transformer
        trans_feat = self.trans(seq)

        if self.mode == 'only':
            return self.head(trans_feat).squeeze()

        # Path B: Auxiliary
        aux_feat = self.aux_mlp(aux_data)

        # Fusion
        combined = torch.cat([trans_feat, aux_feat], dim=1)
        return self.head(combined).squeeze()

# ==============================================================================
# 4. Training Engine
# ==============================================================================
def train_step(model, loader, optimizer, criterion, mode):
    model.train()
    total_loss = 0
    for seqs, descs, fps, labels in loader:
        seqs, labels = seqs.to(DEVICE), labels.to(DEVICE)

        if mode == 'desc': aux = descs.to(DEVICE)
        elif mode == 'fp': aux = fps.to(DEVICE)
        else: aux = None # 'only' mode

        optimizer.zero_grad()
        preds = model(seqs, aux)
        loss = criterion(preds, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item() * labels.size(0)
    return total_loss / len(loader.dataset)

def eval_step(model, loader, mode):
    model.eval()
    preds_all, labels_all = [], []
    with torch.no_grad():
        for seqs, descs, fps, labels in loader:
            seqs = seqs.to(DEVICE)

            if mode == 'desc': aux = descs.to(DEVICE)
            elif mode == 'fp': aux = fps.to(DEVICE)
            else: aux = None

            p = model(seqs, aux)
            preds_all.extend(p.cpu().numpy())
            labels_all.extend(labels.numpy())

    r2 = r2_score(labels_all, preds_all)
    mse = mean_squared_error(labels_all, preds_all)
    return mse, r2, np.array(labels_all), np.array(preds_all)

# ==============================================================================
# 5. Main Execution
# ==============================================================================
def main():
    # 1. Data Prep
    csv_path = '/content/SweetpredDB.csv'
    if not os.path.exists(csv_path):
        print("Generating dummy data for demo...")
        df = pd.DataFrame({'Smiles': ['C']*20 + ['CCO']*20, 'logSw': np.random.rand(40)})
    else:
        df = pd.read_csv(csv_path)
        df.rename(columns=str.lower, inplace=True)
        df.rename(columns={'smiles': 'Smiles', 'logsw': 'logSw'}, inplace=True)

    tokenizer = SMILESTokenizer()
    tokenizer.build_vocab(df['Smiles'].tolist())

    dataset = TransFusionDataset(df, tokenizer)

    # Split
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_ds, test_ds = random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))

    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_ds, batch_size=32, collate_fn=collate_fn)

    # Dimensions
    sample = dataset[0]
    desc_dim = sample['desc'].shape[0] # 9
    fp_dim = sample['fp'].shape[0]     # 1024
    vocab_size = len(tokenizer.vocab)
    pad_id = tokenizer.vocab["<pad>"]

    print(f"\nModel Config: Vocab={vocab_size}, Desc_Dim={desc_dim}, FP_Dim={fp_dim}")

    # 2. Initialize Models
    # Model A: SMILES Only
    model_only = TransFusionModel('only', vocab_size, 0, pad_id).to(DEVICE)
    # Model B: Transformer + Descriptors
    model_desc = TransFusionModel('desc', vocab_size, desc_dim, pad_id).to(DEVICE)
    # Model C: Transformer + Fingerprints
    model_fp = TransFusionModel('fp', vocab_size, fp_dim, pad_id).to(DEVICE)

    # 3. Optimizers
    opt_only = optim.Adam(model_only.parameters(), lr=0.0005)
    opt_desc = optim.Adam(model_desc.parameters(), lr=0.0005)
    opt_fp = optim.Adam(model_fp.parameters(), lr=0.0005)
    criterion = nn.MSELoss()

    # 4. Training Loop
    epochs = 50
    history = {'only_loss': [], 'desc_loss': [], 'fp_loss': [],
               'only_r2': [], 'desc_r2': [], 'fp_r2': []}

    print("\n🚀 Starting Comparative Training (Transformer)...")
    for epoch in range(epochs):
        # Train
        l_only = train_step(model_only, train_loader, opt_only, criterion, 'only')
        l_desc = train_step(model_desc, train_loader, opt_desc, criterion, 'desc')
        l_fp = train_step(model_fp, train_loader, opt_fp, criterion, 'fp')

        # Eval
        _, r2_only, _, _ = eval_step(model_only, test_loader, 'only')
        _, r2_desc, _, _ = eval_step(model_desc, test_loader, 'desc')
        _, r2_fp, _, _ = eval_step(model_fp, test_loader, 'fp')

        history['only_loss'].append(l_only)
        history['desc_loss'].append(l_desc)
        history['fp_loss'].append(l_fp)

        history['only_r2'].append(r2_only)
        history['desc_r2'].append(r2_desc)
        history['fp_r2'].append(r2_fp)

        if (epoch+1) % 5 == 0:
            print(f"Ep {epoch+1:02d} | [Only] R2:{r2_only:.4f} | [Desc] R2:{r2_desc:.4f} | [FP] R2:{r2_fp:.4f}")

    # 5. Visualization
    print("\n📊 Generating Comparison Charts...")
    _, final_r2_only, y_true, y_pred_only = eval_step(model_only, test_loader, 'only')
    _, final_r2_desc, _, y_pred_desc = eval_step(model_desc, test_loader, 'desc')
    _, final_r2_fp, _, y_pred_fp = eval_step(model_fp, test_loader, 'fp')

    fig = plt.figure(figsize=(18, 6))

    # Plot 1: Loss
    ax1 = plt.subplot(1, 3, 1)
    ax1.plot(history['only_loss'], label='SMILES Only', color='gray', linestyle='--')
    ax1.plot(history['desc_loss'], label='SMILES + Desc', color='tab:blue')
    ax1.plot(history['fp_loss'], label='SMILES + FP', color='tab:orange')
    ax1.set_title('Training Loss')
    ax1.set_xlabel('Epoch')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Plot 2: R2 Score
    ax2 = plt.subplot(1, 3, 2)
    ax2.plot(history['only_r2'], label='SMILES Only', color='gray', linestyle='--')
    ax2.plot(history['desc_r2'], label='SMILES + Desc', color='tab:blue')
    ax2.plot(history['fp_r2'], label='SMILES + FP', color='tab:orange')
    ax2.set_title('Test R2 Score')
    ax2.set_xlabel('Epoch')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # Plot 3: Scatter Plot
    ax3 = plt.subplot(1, 3, 3)
    ax3.scatter(y_true, y_pred_only, alpha=0.3, s=15, label=f'Only (R2={final_r2_only:.3f})', color='gray')
    ax3.scatter(y_true, y_pred_desc, alpha=0.5, s=20, label=f'+Desc (R2={final_r2_desc:.3f})', color='tab:blue')
    ax3.scatter(y_true, y_pred_fp, alpha=0.5, s=20, label=f'+FP (R2={final_r2_fp:.3f})', color='tab:orange', marker='x')

    min_v, max_v = y_true.min(), y_true.max()
    ax3.plot([min_v, max_v], [min_v, max_v], 'k--', lw=1)
    ax3.set_title('Final Prediction Correlation')
    ax3.set_xlabel('True Sweetness')
    ax3.set_ylabel('Predicted')
    ax3.legend()
    ax3.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\n🏆 Final Comparison:")
    print(f"   Model A (SMILES Only):        R2 = {final_r2_only:.4f}")
    print(f"   Model B (SMILES + Desc):      R2 = {final_r2_desc:.4f}")
    print(f"   Model C (SMILES + Fingerprints): R2 = {final_r2_fp:.4f}")

if __name__ == '__main__':
    main()

## **SMIELS+Graph**

In [ ]:
import os
import re
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from collections import Counter
from scipy.sparse import coo_matrix
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence

# RDKit & Sklearn
from rdkit import Chem
from rdkit.Chem import rdchem, rdBase
from sklearn.metrics import r2_score, mean_squared_error

# PyG imports
try:
    from torch_geometric.data import Data, Batch
    from torch_geometric.nn import GATConv, global_mean_pool
except ImportError:
    print("Installing PyG dependencies...")
    os.system("pip install -q torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cu118.html")
    os.system("pip install -q torch-geometric")
    from torch_geometric.data import Data, Batch
    from torch_geometric.nn import GATConv, global_mean_pool

# ==============================================================================
# 0. Global Configuration
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')
rdBase.DisableLog('rdApp.warning')

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(42)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {DEVICE}")

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 150

# ==============================================================================
# 1. Data Processing: Tokenizer & Featurizer
# ==============================================================================

# --- 1.1 SMILES Tokenizer (for Transformer) ---
class SMILESTokenizer:
    def __init__(self, max_len=128):
        self.max_len = max_len
        self.pattern =  r"(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"
        self.regex = re.compile(self.pattern)
        self.special_tokens = ["<pad>", "<unk>", "<sos>", "<eos>"]
        self.vocab = {t: i for i, t in enumerate(self.special_tokens)}
        self.unk_token = "<unk>"
        self.sos_token = "<sos>"
        self.eos_token = "<eos>"
        self.pad_token = "<pad>"

    def tokenize(self, smiles):
        return [token for token in self.regex.findall(smiles)]

    def build_vocab(self, smiles_list):
        counter = Counter()
        for s in smiles_list:
            counter.update(self.tokenize(s))
        for token, _ in counter.most_common():
            if token not in self.vocab:
                self.vocab[token] = len(self.vocab)
        return self.vocab

    def encode(self, smiles):
        tokens = self.tokenize(smiles)[:self.max_len - 2]
        ids = [self.vocab.get(t, self.vocab[self.unk_token]) for t in tokens]
        return [self.vocab[self.sos_token]] + ids + [self.vocab[self.eos_token]]

# --- 1.2 Graph Featurizer (for GNN) ---
def one_hot_encoding(value, choices):
    encoding = [0] * (len(choices) + 1)
    index = choices.index(value) if value in choices else -1
    encoding[index] = 1
    return encoding

class MoleculeFeaturizer(object):
    def _atom_featurizer(self, atom):
        # 37 atom types + 1 other
        atomic_numer = [1, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 46, 47, 48, 49, 50, 51, 52, 53]
        return (one_hot_encoding(atom.GetAtomicNum(), atomic_numer) +
                one_hot_encoding(atom.GetTotalDegree(), list(range(5))) +
                one_hot_encoding(int(atom.GetHybridization()), list(range(len(Chem.HybridizationType.names)-1))) +
                one_hot_encoding(atom.GetChiralTag(), list(range(len(Chem.ChiralType.names)-1))) +
                one_hot_encoding(atom.GetTotalNumHs(), list(range(5))) +
                [1 if atom.GetIsAromatic() else 0])

    def _bond_featurizer(self, bond):
        bt = [
            int(bond.GetBondType() == rdchem.BondType.SINGLE),
            int(bond.GetBondType() == rdchem.BondType.DOUBLE),
            int(bond.GetBondType() == rdchem.BondType.TRIPLE),
            int(bond.GetBondType() == rdchem.BondType.AROMATIC)
        ]
        return bt + [int(bond.GetIsConjugated()), int(bond.IsInRing())]

    def __call__(self, mol):
        atom_features = [self._atom_featurizer(atom) for atom in mol.GetAtoms()]
        x = torch.tensor(atom_features, dtype=torch.float32)

        adj = Chem.GetAdjacencyMatrix(mol)
        coo_adj = coo_matrix(adj)
        row, col = coo_adj.row, coo_adj.col
        edge_index = torch.tensor([row, col], dtype=torch.long)

        bond_features = []
        for i, j in zip(row, col):
            bond = mol.GetBondBetweenAtoms(int(i), int(j))
            bond_features.append(self._bond_featurizer(bond))

        edge_attr = torch.tensor(bond_features, dtype=torch.float32) if len(bond_features) > 0 else torch.empty((0, 6), dtype=torch.float32)
        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

# ==============================================================================
# 2. Dataset Construction
# ==============================================================================
class MultimodalDataset(Dataset):
    def __init__(self, df, tokenizer, featurizer):
        self.data = []
        print("Processing data (Graph + Sequence)...")

        for _, row in tqdm(df.iterrows(), total=len(df)):
            mol = Chem.MolFromSmiles(row['Smiles'])
            if mol is None: continue

            # 1. Graph Data
            graph_data = featurizer(mol)

            # 2. Sequence Data
            seq_ids = tokenizer.encode(row['Smiles'])

            self.data.append({
                'graph': graph_data,
                'seq': torch.tensor(seq_ids, dtype=torch.long),
                'y': torch.tensor([row['logSw']], dtype=torch.float32)
            })

    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]

def collate_fn(batch):
    graphs = Batch.from_data_list([item['graph'] for item in batch])

    # Pad Sequence
    pad_id = 0 # Assuming <pad> is at index 0
    seqs = pad_sequence([item['seq'] for item in batch], batch_first=True, padding_value=pad_id)

    labels = torch.cat([item['y'] for item in batch])
    return graphs, seqs, labels

# ==============================================================================
# 3. Model Architectures
# ==============================================================================

# --- 3.1 GNN Encoder (Strictly matches Strategy B parameters) ---
class GNNEncoder(nn.Module):
    def __init__(self, input_dim=69, hidden_dim=16, heads=2, edge_dim=6, dropout=0.4):
        super(GNNEncoder, self).__init__()
        self.gat1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=dropout, edge_dim=edge_dim)
        self.gat2 = GATConv(hidden_dim * heads, hidden_dim, heads=1, dropout=dropout, edge_dim=edge_dim)
        self.pool = global_mean_pool
        self.output_dim = hidden_dim # 16

    def forward(self, x, edge_index, batch, edge_attr):
        x = self.gat1(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)
        x = self.gat2(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)
        x = self.pool(x, batch)
        return x

# --- 3.2 Transformer Encoder (Strictly matches Encoder-only code) ---
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

class TransformerEncoderBranch(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=4, num_layers=3, dim_feedforward=256, pad_idx=0, dropout=0.1):
        super().__init__()
        self.pad_idx = pad_idx
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_encoder = PositionalEncoding(d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead,
            dim_feedforward=dim_feedforward,
            batch_first=True, dropout=dropout
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_dim = d_model # 128

    def forward(self, x):
        padding_mask = (x == self.pad_idx)
        x = self.embedding(x) * math.sqrt(self.output_dim)
        x = self.pos_encoder(x)
        x = self.transformer(x, src_key_padding_mask=padding_mask)

        # Masked Mean Pooling
        mask_expanded = (~padding_mask).unsqueeze(-1).float()
        sum_embeddings = torch.sum(x * mask_expanded, dim=1)
        sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
        feature = sum_embeddings / sum_mask
        return feature

# --- 3.3 Universal Model Wrapper (Single / Dual) ---
class SweetNetModel(nn.Module):
    def __init__(self, mode, gnn_cfg, trans_cfg):
        """
        mode: 'graph', 'seq', 'fusion'
        """
        super().__init__()
        self.mode = mode

        # Initialize Branches
        self.gnn = GNNEncoder(**gnn_cfg) if mode in ['graph', 'fusion'] else None
        self.trans = TransformerEncoderBranch(**trans_cfg) if mode in ['seq', 'fusion'] else None

        # Determine Fusion Dimension
        fusion_dim = 0
        if self.gnn: fusion_dim += self.gnn.output_dim # 16
        if self.trans: fusion_dim += self.trans.output_dim # 128

        # Prediction Head (MLP) - Consistent with previous code
        self.mlp = nn.Sequential(
            nn.Linear(fusion_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3), # Combined dropout
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )

        # Init weights
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)

    def forward(self, graph_batch, seq_batch):
        feats = []

        if self.gnn:
            g_emb = self.gnn(graph_batch.x, graph_batch.edge_index, graph_batch.batch, graph_batch.edge_attr)
            feats.append(g_emb)

        if self.trans:
            s_emb = self.trans(seq_batch)
            feats.append(s_emb)

        # Concatenate Features
        combined = torch.cat(feats, dim=1)
        return self.mlp(combined).squeeze()

# ==============================================================================
# 4. Training Engine (Standardized)
# ==============================================================================
def train_step(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for graphs, seqs, labels in loader:
        graphs, seqs, labels = graphs.to(DEVICE), seqs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        preds = model(graphs, seqs)
        loss = criterion(preds, labels)
        loss.backward()

        # Gradient Clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item() * labels.size(0)
    return total_loss / len(loader.dataset)

def eval_step(model, loader):
    model.eval()
    preds_all, labels_all = [], []
    with torch.no_grad():
        for graphs, seqs, labels in loader:
            graphs, seqs, labels = graphs.to(DEVICE), seqs.to(DEVICE), labels.to(DEVICE)

            p = model(graphs, seqs)
            preds_all.extend(p.cpu().numpy())
            labels_all.extend(labels.cpu().numpy())

    mse = mean_squared_error(labels_all, preds_all)
    r2 = r2_score(labels_all, preds_all)
    return mse, r2, np.array(labels_all), np.array(preds_all)

def run_experiment(model_name, mode, gnn_cfg, trans_cfg, train_loader, val_loader, test_loader, epochs=200):
    print(f"\n🚀 Training Model: [{model_name}] (Mode: {mode})")

    # Init Model
    model = SweetNetModel(mode, gnn_cfg, trans_cfg).to(DEVICE)

    # Optimizer (Use weight_decay=2e-3 for GNN consistency, adjusted slightly for Transformer stability)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4) # Balanced decay
    criterion = nn.MSELoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)

    # Training Loop
    best_val_r2 = -float('inf')
    best_wts = copy.deepcopy(model.state_dict())
    history = {'loss': [], 'val_r2': []}

    for epoch in range(epochs):
        loss = train_step(model, train_loader, optimizer, criterion)
        val_mse, val_r2, _, _ = eval_step(model, val_loader)

        scheduler.step(val_mse)
        history['loss'].append(loss)
        history['val_r2'].append(val_r2)

        if val_r2 > best_val_r2:
            best_val_r2 = val_r2
            best_wts = copy.deepcopy(model.state_dict())

        if (epoch+1) % 20 == 0:
            print(f"   Ep {epoch+1:03d} | Loss: {loss:.4f} | Val R2: {val_r2:.4f}")

    # Final Test Evaluation
    model.load_state_dict(best_wts)
    _, test_r2, y_true, y_pred = eval_step(model, test_loader)
    print(f"   🏆 Final Test R2: {test_r2:.4f}")

    return test_r2, history, y_true, y_pred

# ==============================================================================
# 5. Main Execution
# ==============================================================================
def main():
    # 1. Data Prep
    csv_path = '/content/SweetpredDB.csv'
    if not os.path.exists(csv_path):
        print("Generating dummy data for demo...")
        df = pd.DataFrame({'Smiles': ['C']*50 + ['CC']*50, 'logSw': np.random.rand(100)})
    else:
        df = pd.read_csv(csv_path)
        df.rename(columns=str.lower, inplace=True)
        df.rename(columns={'smiles': 'Smiles', 'logsw': 'logSw'}, inplace=True)

    tokenizer = SMILESTokenizer()
    tokenizer.build_vocab(df['Smiles'].tolist())
    featurizer = MoleculeFeaturizer()

    dataset = MultimodalDataset(df, tokenizer, featurizer)

    # 8:1:1 Split
    train_len = int(0.8 * len(dataset))
    val_len = int(0.1 * len(dataset))
    test_len = len(dataset) - train_len - val_len
    train_ds, val_ds, test_ds = random_split(dataset, [train_len, val_len, test_len], generator=torch.Generator().manual_seed(42))

    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, collate_fn=collate_fn)

    # Model Configs
    gnn_config = {
        'input_dim': 69, 'hidden_dim': 16, 'heads': 2, 'edge_dim': 6, 'dropout': 0.4
    }
    trans_config = {
        'vocab_size': len(tokenizer.vocab), 'd_model': 128, 'nhead': 4,
        'num_layers': 3, 'dim_feedforward': 256, 'pad_idx': tokenizer.vocab["<pad>"], 'dropout': 0.1
    }

    # 2. Run Comparisons
    results = {}

    # Experiment A: Graph Only
    r2_g, hist_g, y_g, pred_g = run_experiment("Graph Only", 'graph', gnn_config, trans_config, train_loader, val_loader, test_loader)

    # Experiment B: SMILES Only
    r2_s, hist_s, y_s, pred_s = run_experiment("SMILES Only", 'seq', gnn_config, trans_config, train_loader, val_loader, test_loader)

    # Experiment C: Graph + SMILES Fusion
    r2_f, hist_f, y_f, pred_f = run_experiment("Graph + SMILES", 'fusion', gnn_config, trans_config, train_loader, val_loader, test_loader)

    # 3. Visualization
    fig = plt.figure(figsize=(18, 6))

    # Plot 1: Validation R2 Curves
    ax1 = plt.subplot(1, 3, 1)
    ax1.plot(hist_g['val_r2'], label='Graph Only', color='tab:blue')
    ax1.plot(hist_s['val_r2'], label='SMILES Only', color='tab:orange')
    ax1.plot(hist_f['val_r2'], label='Fusion (Graph+SMILES)', color='tab:green', linewidth=2)
    ax1.set_title('Validation R2 History')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('R2 Score')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Plot 2: Bar Chart
    ax2 = plt.subplot(1, 3, 2)
    labels = ['Graph Only', 'SMILES Only', 'Fusion']
    values = [r2_g, r2_s, r2_f]
    colors = ['tab:blue', 'tab:orange', 'tab:green']
    bars = ax2.bar(labels, values, color=colors)
    ax2.set_title('Final Test R2 Score Comparison')
    ax2.set_ylim(0, 1.0)
    ax2.grid(axis='y', alpha=0.3)
    for bar in bars:
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f"{bar.get_height():.3f}", ha='center', va='bottom', fontweight='bold')

    # Plot 3: Scatter Plot (Fusion)
    ax3 = plt.subplot(1, 3, 3)
    ax3.scatter(y_f, pred_f, alpha=0.5, color='tab:green', s=20, label=f'Fusion (R2={r2_f:.3f})')
    # Add light references for others
    # ax3.scatter(y_g, pred_g, alpha=0.1, color='tab:blue', s=10)

    min_v, max_v = y_f.min(), y_f.max()
    ax3.plot([min_v, max_v], [min_v, max_v], 'k--', lw=1)
    ax3.set_title('Fusion Model Predictions')
    ax3.set_xlabel('True Sweetness')
    ax3.set_ylabel('Predicted')
    ax3.legend()
    ax3.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print("\n" + "="*40)
    print("🏆 SUMMARY REPORT")
    print("="*40)
    print(f"Graph Only R2:      {r2_g:.4f}")
    print(f"SMILES Only R2:     {r2_s:.4f}")
    print(f"Fusion Model R2:    {r2_f:.4f}")

if __name__ == '__main__':
    main()

# **第2部分：再试试另一种二模态：Graph/SMILES+预训练特征**

In [ ]:
import os
import re
import json
import math
import random
import warnings
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from collections import Counter
from scipy.sparse import coo_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence

# RDKit & Sklearn
from rdkit import Chem
from rdkit.Chem import rdchem, rdBase, AllChem, Descriptors
from sklearn.metrics import r2_score, mean_squared_error
from transformers import AutoTokenizer, AutoModel

# PyG imports
try:
    from torch_geometric.data import Data, Batch
    from torch_geometric.nn import GATConv, global_mean_pool
except ImportError:
    print("Installing PyG dependencies...")
    os.system("pip install -q torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cu118.html")
    os.system("pip install -q torch-geometric")
    from torch_geometric.data import Data, Batch
    from torch_geometric.nn import GATConv, global_mean_pool

# Configuration
warnings.filterwarnings('ignore')
rdBase.DisableLog('rdApp.warning')
logging.getLogger("transformers").setLevel(logging.ERROR)

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(42)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using Device: {DEVICE}")

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 150

# ==============================================================================
# 1. Data Processing Components
# ==============================================================================

# --- 1.1 Tokenizer ---
class SMILESTokenizer:
    def __init__(self, max_len=128):
        self.max_len = max_len
        self.pattern =  r"(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"
        self.regex = re.compile(self.pattern)
        self.special_tokens = ["<pad>", "<unk>", "<sos>", "<eos>"]
        self.vocab = {t: i for i, t in enumerate(self.special_tokens)}

    def tokenize(self, smiles):
        return [token for token in self.regex.findall(smiles)]

    def build_vocab(self, smiles_list):
        counter = Counter()
        for s in smiles_list:
            counter.update(self.tokenize(s))
        for token, _ in counter.most_common():
            if token not in self.vocab:
                self.vocab[token] = len(self.vocab)
        return self.vocab

    def encode(self, smiles):
        tokens = self.tokenize(smiles)[:self.max_len - 2]
        ids = [self.vocab.get(t, self.vocab["<unk>"]) for t in tokens]
        return [self.vocab["<sos>"]] + ids + [self.vocab["<eos>"]]

# --- 1.2 Graph Featurizer ---
def one_hot_encoding(value, choices):
    encoding = [0] * (len(choices) + 1)
    index = choices.index(value) if value in choices else -1
    encoding[index] = 1
    return encoding

class MoleculeFeaturizer:
    def _atom_featurizer(self, atom):
        atomic_numer = [1, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 46, 47, 48, 49, 50, 51, 52, 53]
        return (one_hot_encoding(atom.GetAtomicNum(), atomic_numer) +
                one_hot_encoding(atom.GetTotalDegree(), list(range(5))) +
                one_hot_encoding(int(atom.GetHybridization()), list(range(len(Chem.HybridizationType.names)-1))) +
                one_hot_encoding(atom.GetChiralTag(), list(range(len(Chem.ChiralType.names)-1))) +
                one_hot_encoding(atom.GetTotalNumHs(), list(range(5))) +
                [1 if atom.GetIsAromatic() else 0])

    def _bond_featurizer(self, bond):
        bt = [int(bond.GetBondType() == t) for t in [rdchem.BondType.SINGLE, rdchem.BondType.DOUBLE, rdchem.BondType.TRIPLE, rdchem.BondType.AROMATIC]]
        return bt + [int(bond.GetIsConjugated()), int(bond.IsInRing())]

    def __call__(self, mol):
        atom_features = [self._atom_featurizer(atom) for atom in mol.GetAtoms()]
        x = torch.tensor(atom_features, dtype=torch.float32)

        adj = Chem.GetAdjacencyMatrix(mol)
        coo_adj = coo_matrix(adj)
        row, col = coo_adj.row, coo_adj.col
        edge_index = torch.tensor([row, col], dtype=torch.long)

        bond_features = []
        for i, j in zip(row, col):
            bond = mol.GetBondBetweenAtoms(int(i), int(j))
            bond_features.append(self._bond_featurizer(bond))

        edge_attr = torch.tensor(bond_features, dtype=torch.float32) if bond_features else torch.empty((0, 6), dtype=torch.float32)
        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

# --- 1.3 Pretrained Featurizer ---
class PretrainFeaturizer:
    def __init__(self):
        model_name = "seyonec/ChemBERTa-zinc-base-v1"
        print(f"🤖 Loading ChemBERTa ({model_name})...")
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModel.from_pretrained(model_name).to(DEVICE).eval()
            self.active = True
        except Exception as e:
            print(f"❌ Failed to load ChemBERTa: {e}")
            self.active = False

    def __call__(self, smiles):
        if not self.active: return torch.randn(768)
        with torch.no_grad():
            inputs = self.tokenizer(smiles, return_tensors="pt", padding=True, truncation=True, max_length=128)
            inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
            outputs = self.model(**inputs)
            return outputs.last_hidden_state[0, 0, :].cpu() # CLS token

# ==============================================================================
# 2. Dataset & Dataloader
# ==============================================================================
class HybridDataset(Dataset):
    def __init__(self, df, smiles_tokenizer, save_path="hybrid_data.pt"):
        self.data = []

        if os.path.exists(save_path):
            print(f"🔄 Loading cached data from {save_path}...")
            self.data = torch.load(save_path)
        else:
            print("⚡ Processing data (Graph + Sequence + Pretrained)...")
            graph_featurizer = MoleculeFeaturizer()
            pretrain_featurizer = PretrainFeaturizer()

            for _, row in tqdm(df.iterrows(), total=len(df)):
                mol = Chem.MolFromSmiles(row['Smiles'])
                if mol is None: continue

                self.data.append({
                    'graph': graph_featurizer(mol),
                    'seq': torch.tensor(smiles_tokenizer.encode(row['Smiles']), dtype=torch.long),
                    'pretrain': pretrain_featurizer(row['Smiles']),
                    'y': torch.tensor([row['logSw']], dtype=torch.float32)
                })
            torch.save(self.data, save_path)
            print("✅ Data processed and saved.")

    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]

def collate_fn(batch):
    graphs = Batch.from_data_list([x['graph'] for x in batch])
    seqs = pad_sequence([x['seq'] for x in batch], batch_first=True, padding_value=0)
    pretrains = torch.stack([x['pretrain'] for x in batch])
    labels = torch.cat([x['y'] for x in batch])
    return graphs, seqs, pretrains, labels

# ==============================================================================
# 3. Model Architectures
# ==============================================================================

# --- Tower 1: Graph (GAT) ---
class GraphTower(nn.Module):
    def __init__(self, input_dim=69, hidden_dim=16, heads=2, edge_dim=6, dropout=0.4):
        super().__init__()
        self.gat1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=dropout, edge_dim=edge_dim)
        self.gat2 = GATConv(hidden_dim * heads, hidden_dim, heads=1, dropout=dropout, edge_dim=edge_dim)
        self.pool = global_mean_pool
        self.out_dim = hidden_dim

    def forward(self, data):
        x, edge_index, batch, edge_attr = data.x, data.edge_index, data.batch, data.edge_attr
        x = self.gat1(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)
        x = self.gat2(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)
        return self.pool(x, batch)

# --- Tower 2: Sequence (Transformer) ---
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))
    def forward(self, x): return x + self.pe[:, :x.size(1), :]

class SequenceTower(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=4, num_layers=3, pad_idx=0):
        super().__init__()
        self.pad_idx = pad_idx
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos = PositionalEncoding(d_model)
        enc_layer = nn.TransformerEncoderLayer(d_model, nhead, 256, batch_first=True, dropout=0.1)
        self.transformer = nn.TransformerEncoder(enc_layer, num_layers)
        self.out_dim = d_model

    def forward(self, x):
        mask = (x == self.pad_idx)
        x = self.embedding(x) * math.sqrt(self.out_dim)
        x = self.pos(x)
        x = self.transformer(x, src_key_padding_mask=mask)
        # Mean pooling ignoring padding
        mask_expanded = (~mask).unsqueeze(-1).float()
        return (x * mask_expanded).sum(1) / mask_expanded.sum(1).clamp(min=1e-9)

# --- Tower 3: Pretrained (MLP Projection) ---
class PretrainTower(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        self.out_dim = hidden_dim

    def forward(self, x):
        return self.net(x)

# --- Fusion Model ---
class FusionModel(nn.Module):
    def __init__(self, mode, vocab_size, pad_idx):
        """
        mode: 'graph', 'seq', 'graph+pre', 'seq+pre'
        """
        super().__init__()
        self.mode = mode

        # Initialize Towers based on mode
        self.graph = GraphTower() if 'graph' in mode else None
        self.seq = SequenceTower(vocab_size, pad_idx=pad_idx) if 'seq' in mode else None
        self.pre = PretrainTower() if 'pre' in mode else None

        # Calculate fusion dimension
        fusion_dim = 0
        if self.graph: fusion_dim += self.graph.out_dim # 16
        if self.seq: fusion_dim += self.seq.out_dim # 128
        if self.pre: fusion_dim += self.pre.out_dim # 128

        self.head = nn.Sequential(
            nn.Linear(fusion_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)
        )

    def forward(self, graph_data, seq_data, pre_data):
        feats = []
        if self.graph: feats.append(self.graph(graph_data))
        if self.seq: feats.append(self.seq(seq_data))
        if self.pre: feats.append(self.pre(pre_data))

        combined = torch.cat(feats, dim=1)
        return self.head(combined).squeeze()

# ==============================================================================
# 4. Training Engine
# ==============================================================================
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for graphs, seqs, pres, labels in loader:
        graphs = graphs.to(DEVICE)
        seqs = seqs.to(DEVICE)
        pres = pres.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()
        preds = model(graphs, seqs, pres)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * labels.size(0)
    return total_loss / len(loader.dataset)

def evaluate(model, loader):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for graphs, seqs, pres, labels in loader:
            graphs = graphs.to(DEVICE)
            seqs = seqs.to(DEVICE)
            pres = pres.to(DEVICE)

            p = model(graphs, seqs, pres)
            preds.extend(p.cpu().numpy())
            targets.extend(labels.numpy())
    return r2_score(targets, preds), np.array(targets), np.array(preds)

# ==============================================================================
# 5. Main Comparison Loop
# ==============================================================================
def main():
    # 1. Load Data
    csv_path = 'SweetpredDB.csv'
    if not os.path.exists(csv_path):
        print("Creating dummy data...")
        df = pd.DataFrame({'Smiles': ['C']*20 + ['CC']*20, 'logSw': np.random.rand(40)})
    else:
        df = pd.read_csv(csv_path)
        df.rename(columns=str.lower, inplace=True)
        df.rename(columns={'smiles': 'Smiles', 'logsw': 'logSw'}, inplace=True)

    tokenizer = SMILESTokenizer()
    tokenizer.build_vocab(df['Smiles'].tolist())

    # 2. Prepare Dataset
    dataset = HybridDataset(df, tokenizer)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_ds, test_ds = random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))

    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_ds, batch_size=32, collate_fn=collate_fn)

    vocab_size = len(tokenizer.vocab)
    pad_id = tokenizer.vocab["<pad>"]

    # 3. Define Configurations
    configs = {
        'Graph Only': 'graph',
        'SMILES Only': 'seq',
        'Graph + Pretrain': 'graph+pre',
        'SMILES + Pretrain': 'seq+pre'
    }

    results = {}
    histories = {}

    # 4. Train All Models
    print(f"\n🚀 Starting Comparison of {len(configs)} Models...")

    for name, mode in configs.items():
        print(f"\nTraining [{name}]...")
        model = FusionModel(mode, vocab_size, pad_id).to(DEVICE)
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()

        loss_hist = []
        for epoch in range(50):
            loss = train_epoch(model, train_loader, optimizer, criterion)
            loss_hist.append(loss)
            if (epoch+1) % 10 == 0:
                r2, _, _ = evaluate(model, test_loader)
                print(f"  Ep {epoch+1:02d} | Loss: {loss:.4f} | Test R2: {r2:.4f}")

        final_r2, y_true, y_pred = evaluate(model, test_loader)
        results[name] = final_r2
        histories[name] = loss_hist

    # 5. Visualization
    print("\n🏆 Final Results:")
    for k, v in results.items():
        print(f"{k:<20}: R2 = {v:.4f}")

    fig = plt.figure(figsize=(14, 6))

    # Loss Curves
    ax1 = plt.subplot(1, 2, 1)
    for name, hist in histories.items():
        ax1.plot(hist, label=name)
    ax1.set_title("Training Loss Curves")
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("MSE Loss")
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Performance Bar Chart
    ax2 = plt.subplot(1, 2, 2)
    colors = ['gray', 'gray', '#1f77b4', '#ff7f0e'] # Highlight multimodal
    bars = ax2.bar(results.keys(), results.values(), color=colors)
    ax2.set_title("Test Set R2 Score Comparison")
    ax2.set_ylim(0, 1.0)
    ax2.grid(axis='y', alpha=0.3)

    for bar in bars:
        yval = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2, yval, f"{yval:.3f}", va='bottom', ha='center', fontweight='bold')

    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    main()

### **看看预训练特征长啥样**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd

# ==========================================
# 1. 准备数据
# ==========================================
# 随便找 3 个分子：甲烷、乙酸、苯
smiles_samples = [
    "C",              # 很短
    "CC(=O)O",        # 中等
    "c1ccccc1"        # 苯环
]

print(f"🧪 原始输入数据 (List of SMILES): {smiles_samples}")
print("-" * 60)

# ==========================================
# 2. 加载预训练模型 (ChemBERTa)
# ==========================================
# 这是一个在数百万化学分子上预训练过的模型
model_name = "seyonec/ChemBERTa-zinc-base-v1"
print(f"🤖 正在加载模型: {model_name} ...")

try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    print("✅ 加载成功！")
except Exception as e:
    print(f"❌ 加载失败 (可能网络问题): {e}")
    # 为了演示，如果下载失败，我们就不继续运行模型部分了
    model = None

if model:
    # ==========================================
    # 3. Tokenizer 处理 (文本 -> 数字ID)
    # ==========================================
    # padding=True: 把短的分子补齐，保证一个batch里长度一样
    # return_tensors="pt": 返回 PyTorch 的 Tensor 格式
    inputs = tokenizer(smiles_samples, padding=True, truncation=True, return_tensors="pt")

    print("\n🔍 [Step 1] Tokenizer 处理后的数据:")
    print(f"Input IDs (形状 {inputs['input_ids'].shape}):\n{inputs['input_ids']}")
    print(f"\nAttention Mask (形状 {inputs['attention_mask'].shape}):\n{inputs['attention_mask']}")
    print("(注意看：1 代表有效token，0 代表补齐的padding)")

    # ==========================================
    # 4. 模型前向传播 (数字ID -> 高维向量)
    # ==========================================
    print("\n⚡ [Step 2] 输入模型进行计算...")
    with torch.no_grad():
        outputs = model(**inputs)

    # 提取核心输出
    # last_hidden_state: 每一个 Token 对应的向量
    # pooler_output (或者取第一个token CLS): 整个分子的代表向量

    token_embeddings = outputs.last_hidden_state
    cls_embedding = token_embeddings[:, 0, :] # 取每个句子的第一个token (CLS) 作为整体特征

    print("-" * 60)
    print("📊 [Step 3] 最终输出结果解析:")

    print(f"\n1. 完整序列特征 (Last Hidden State):")
    print(f"   形状: {token_embeddings.shape}")
    print(f"   解读: [Batch Size (3), 序列长度 ({token_embeddings.shape[1]}), 特征维度 (768)]")
    print("   含义: 每个原子/字符都被转换成了一个 768 维的向量。")

    print(f"\n2. 分子整体特征 (CLS Token Embedding):")
    print(f"   形状: {cls_embedding.shape}")
    print(f"   解读: [Batch Size (3), 特征维度 (768)]")
    print("   含义: 这就是我们通常用来做‘下游任务’（如甜度预测）的那个融合特征向量。")

    print(f"\n3. 看看第一个分子 ('C') 的前10位特征数值:")
    print(f"   {cls_embedding[0][:10].numpy()}")

# **第2部分：试试更多模态（Graph+SMILES+指纹&描述符+预训练特征）**

In [ ]:
import os
import re
import math
import json
import random
import pickle  # 用于保存字典数据
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from collections import Counter
from scipy.sparse import coo_matrix
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset

# RDKit & Sklearn
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, rdBase, rdchem
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split # 使用 sklearn 进行更可控的划分

# PyG & Transformers
try:
    from torch_geometric.data import Data, Batch
    from torch_geometric.nn import GATConv, global_mean_pool
    from transformers import AutoTokenizer, AutoModel
except ImportError:
    print("正在安装依赖库...")
    os.system("pip install -q torch-geometric transformers")
    from torch_geometric.data import Data, Batch
    from torch_geometric.nn import GATConv, global_mean_pool
    from transformers import AutoTokenizer, AutoModel

# ==============================================================================
# 0. 全局配置与目录设置
# ==============================================================================
import warnings
import logging
warnings.filterwarnings('ignore')
rdBase.DisableLog('rdApp.warning')
logging.getLogger("transformers").setLevel(logging.ERROR)

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(42)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using Device: {DEVICE}")

# 创建保存目录
SAVE_DIR = "experiment_results"
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)
    print(f"📂 Created directory: {SAVE_DIR}")

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 150

# ==============================================================================
# 1. 特征工厂 (Feature Factory)
# ==============================================================================

class SMILESTokenizer:
    def __init__(self, max_len=128):
        self.max_len = max_len
        self.pattern =  r"(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"
        self.regex = re.compile(self.pattern)
        self.special_tokens = ["<pad>", "<unk>", "<sos>", "<eos>"]
        self.vocab = {t: i for i, t in enumerate(self.special_tokens)}

    def tokenize(self, smiles):
        return [token for token in self.regex.findall(smiles)]

    def build_vocab(self, smiles_list):
        counter = Counter()
        for s in smiles_list:
            counter.update(self.tokenize(s))
        for token, _ in counter.most_common():
            if token not in self.vocab:
                self.vocab[token] = len(self.vocab)
        return self.vocab

    def encode(self, smiles):
        tokens = self.tokenize(smiles)[:self.max_len - 2]
        ids = [self.vocab.get(t, self.vocab["<unk>"]) for t in tokens]
        return [self.vocab["<sos>"]] + ids + [self.vocab["<eos>"]]

def one_hot_encoding(value, choices):
    encoding = [0] * (len(choices) + 1)
    index = choices.index(value) if value in choices else -1
    encoding[index] = 1
    return encoding

class MoleculeFeaturizer:
    def _atom_featurizer(self, atom):
        atomic_numer = [1, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 46, 47, 48, 49, 50, 51, 52, 53]
        return (one_hot_encoding(atom.GetAtomicNum(), atomic_numer) +
                one_hot_encoding(atom.GetTotalDegree(), list(range(5))) +
                one_hot_encoding(int(atom.GetHybridization()), list(range(len(Chem.HybridizationType.names)-1))) +
                one_hot_encoding(atom.GetChiralTag(), list(range(len(Chem.ChiralType.names)-1))) +
                one_hot_encoding(atom.GetTotalNumHs(), list(range(5))) +
                [1 if atom.GetIsAromatic() else 0])

    def _bond_featurizer(self, bond):
        bt = [int(bond.GetBondType() == t) for t in [rdchem.BondType.SINGLE, rdchem.BondType.DOUBLE, rdchem.BondType.TRIPLE, rdchem.BondType.AROMATIC]]
        return bt + [int(bond.GetIsConjugated()), int(bond.IsInRing())]

    def __call__(self, mol):
        atom_features = [self._atom_featurizer(atom) for atom in mol.GetAtoms()]
        x = torch.tensor(atom_features, dtype=torch.float32)

        adj = Chem.GetAdjacencyMatrix(mol)
        coo_adj = coo_matrix(adj)
        row, col = coo_adj.row, coo_adj.col
        edge_index = torch.tensor([row, col], dtype=torch.long)

        bond_features = []
        for i in range(len(row)):
            u, v = int(row[i]), int(col[i])
            bond = mol.GetBondBetweenAtoms(u, v)
            if bond is not None:
                bond_features.append(self._bond_featurizer(bond))
            else:
                bond_features.append([0]*6)

        if len(bond_features) > 0:
            edge_attr = torch.tensor(bond_features, dtype=torch.float32)
        else:
            edge_attr = torch.empty((0, 6), dtype=torch.float32)

        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

class AdvancedFeaturizer:
    def __init__(self):
        self.model_name = "seyonec/ChemBERTa-zinc-base-v1"
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModel.from_pretrained(self.model_name).to(DEVICE).eval()
            self.has_bert = True
            print("✅ ChemBERTa Loaded.")
        except Exception as e:
            print(f"⚠️ ChemBERTa load failed: {e}")
            self.has_bert = False

    def get_bert_feature(self, smiles):
        if not self.has_bert: return torch.randn(768)
        with torch.no_grad():
            inputs = self.tokenizer(smiles, return_tensors="pt", padding=True, truncation=True, max_length=128)
            inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
            outputs = self.model(**inputs)
            return outputs.last_hidden_state[0, 0, :].cpu()

    def get_expert_features(self, mol):
        desc = [Descriptors.MolWt(mol), Descriptors.MolLogP(mol), Descriptors.NumHDonors(mol),
                Descriptors.NumHAcceptors(mol), Descriptors.TPSA(mol), Descriptors.NumRotatableBonds(mol),
                Descriptors.FractionCSP3(mol), Descriptors.HallKierAlpha(mol), Descriptors.RingCount(mol)]
        fp = list(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024))
        return torch.tensor(desc + fp, dtype=torch.float32)

# ==============================================================================
# 2. Universal Dataset & Data Splitting
# ==============================================================================
class UniversalDataset(Dataset):
    def __init__(self, df, smiles_tokenizer, save_path=None):
        self.data = []
        # 保存原始 SMILES 以便后续分析
        self.raw_smiles = df['Smiles'].tolist()
        self.raw_labels = df['logSw'].tolist()

        if save_path and os.path.exists(save_path):
            print(f"🔄 Loading cached data from {save_path}...")
            self.data = torch.load(save_path)
        else:
            print("⚡ Generating ALL features (Graph, Seq, Desc, FP, BERT)...")
            graph_featurizer = MoleculeFeaturizer()
            adv_featurizer = AdvancedFeaturizer()

            expert_feats_list = []

            for i, row in tqdm(df.iterrows(), total=len(df)):
                mol = Chem.MolFromSmiles(row['Smiles'])
                if mol is None:
                    # 保持索引一致，无效分子填 None，后续处理
                    self.data.append(None)
                    continue

                expert_f = adv_featurizer.get_expert_features(mol)
                expert_feats_list.append(expert_f)

                self.data.append({
                    'graph': graph_featurizer(mol),
                    'seq': torch.tensor(smiles_tokenizer.encode(row['Smiles']), dtype=torch.long),
                    'expert': expert_f,
                    'bert': adv_featurizer.get_bert_feature(row['Smiles']),
                    'y': torch.tensor([row['logSw']], dtype=torch.float32),
                    'index': i # 保存原始索引
                })

            # Filter None data but keep track of indices mapping if needed
            # 这里简单起见，我们移除 None，注意 self.raw_smiles 对应关系会变
            # 更好的做法是重新对齐 raw_smiles
            valid_indices = [i for i, x in enumerate(self.data) if x is not None]
            self.data = [self.data[i] for i in valid_indices]
            self.raw_smiles = [self.raw_smiles[i] for i in valid_indices]
            self.raw_labels = [self.raw_labels[i] for i in valid_indices]

            # Normalize Expert Features
            if expert_feats_list:
                all_experts = torch.stack(expert_feats_list)
                mean = all_experts.mean(dim=0)
                std = all_experts.std(dim=0) + 1e-6
                for d in self.data:
                    d['expert'] = (d['expert'] - mean) / std

            if save_path:
                torch.save(self.data, save_path)
                print("✅ Data saved.")

    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]

def universal_collate(batch):
    graphs = Batch.from_data_list([x['graph'] for x in batch])
    seqs = pad_sequence([x['seq'] for x in batch], batch_first=True, padding_value=0)
    experts = torch.stack([x['expert'] for x in batch])
    berts = torch.stack([x['bert'] for x in batch])
    labels = torch.cat([x['y'] for x in batch])
    return graphs, seqs, experts, berts, labels

# ==============================================================================
# 3. Model Architecture
# ==============================================================================
# (Same model classes as before)
class GNNBranch(nn.Module):
    def __init__(self, input_dim=69, hidden_dim=16, heads=2, edge_dim=6, dropout=0.4):
        super().__init__()
        self.gat1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=dropout, edge_dim=edge_dim)
        self.gat2 = GATConv(hidden_dim * heads, hidden_dim, heads=1, dropout=dropout, edge_dim=edge_dim)
        self.pool = global_mean_pool
        self.out_dim = hidden_dim
    def forward(self, x, edge_index, batch, edge_attr):
        x = self.gat1(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)
        x = self.gat2(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)
        return self.pool(x, batch)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div)
        pe[:, 1::2] = torch.cos(position * div)
        self.register_buffer('pe', pe.unsqueeze(0))
    def forward(self, x): return x + self.pe[:, :x.size(1), :]

class TransformerBranch(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=4, num_layers=3, pad_idx=0):
        super().__init__()
        self.pad_idx = pad_idx
        self.emb = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos = PositionalEncoding(d_model)
        enc = nn.TransformerEncoderLayer(d_model, nhead, 256, batch_first=True, dropout=0.1)
        self.trans = nn.TransformerEncoder(enc, num_layers)
        self.out_dim = d_model
    def forward(self, x):
        mask = (x == self.pad_idx)
        x = self.pos(self.emb(x) * math.sqrt(self.out_dim))
        x = self.trans(x, src_key_padding_mask=mask)
        mask_exp = (~mask).unsqueeze(-1).float()
        return (x * mask_exp).sum(1) / mask_exp.sum(1).clamp(min=1e-9)

class Projector(nn.Module):
    def __init__(self, input_dim, output_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        self.out_dim = output_dim
    def forward(self, x): return self.net(x)

class FlexibleSweetNet(nn.Module):
    def __init__(self, config, vocab_size, pad_idx):
        super().__init__()
        self.cfg = config
        self.gnn = GNNBranch() if config['use_graph'] else None
        self.trans = TransformerBranch(vocab_size, pad_idx=pad_idx) if config['use_seq'] else None
        self.expert = Projector(1033) if config['use_expert'] else None
        self.bert = Projector(768) if config['use_bert'] else None

        fusion_dim = 0
        if self.gnn: fusion_dim += self.gnn.out_dim
        if self.trans: fusion_dim += self.trans.out_dim
        if self.expert: fusion_dim += self.expert.out_dim
        if self.bert: fusion_dim += self.bert.out_dim

        self.head = nn.Sequential(
            nn.Linear(fusion_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, graphs, seqs, experts, berts):
        feats = []
        if self.gnn: feats.append(self.gnn(graphs.x, graphs.edge_index, graphs.batch, graphs.edge_attr))
        if self.trans: feats.append(self.trans(seqs))
        if self.expert: feats.append(self.expert(experts))
        if self.bert: feats.append(self.bert(berts))
        combined = torch.cat(feats, dim=1)
        return self.head(combined).squeeze()

# ==============================================================================
# 4. Training Engine
# ==============================================================================
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for g, s, e, b, y in loader:
        g, s, e, b, y = g.to(DEVICE), s.to(DEVICE), e.to(DEVICE), b.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        preds = model(g, s, e, b)
        loss = criterion(preds, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item() * y.size(0)
    return total_loss / len(loader.dataset)

def evaluate(model, loader):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for g, s, e, b, y in loader:
            g, s, e, b, y = g.to(DEVICE), s.to(DEVICE), e.to(DEVICE), b.to(DEVICE), y.to(DEVICE)
            p = model(g, s, e, b)
            preds.extend(p.cpu().numpy())
            targets.extend(y.cpu().numpy())
    return r2_score(targets, preds), np.array(targets), np.array(preds)

def run_experiment(name, config, vocab_size, pad_id, loaders, epochs=100):
    print(f"\n🚀 Training [{name}]...")
    train_loader, val_loader, test_loader = loaders

    model = FlexibleSweetNet(config, vocab_size, pad_id).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    criterion = nn.MSELoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=10)

    best_val_r2 = -float('inf')
    best_wts = copy.deepcopy(model.state_dict())
    loss_hist, r2_hist = [], []

    for epoch in range(epochs):
        loss = train_epoch(model, train_loader, optimizer, criterion)
        val_r2, _, _ = evaluate(model, val_loader)
        scheduler.step(loss)
        loss_hist.append(loss)
        r2_hist.append(val_r2)

        if val_r2 > best_val_r2:
            best_val_r2 = val_r2
            best_wts = copy.deepcopy(model.state_dict())

        if (epoch+1) % 20 == 0:
            print(f"   Ep {epoch+1:03d} | Loss: {loss:.4f} | Val R2: {val_r2:.4f}")

    # Save Best Checkpoint
    safe_name = name.replace(" ", "_").replace("+", "_")
    ckpt_path = os.path.join(SAVE_DIR, f"model_{safe_name}.pt")
    torch.save(best_wts, ckpt_path)
    print(f"   💾 Checkpoint saved: {ckpt_path}")

    # Final Test
    model.load_state_dict(best_wts)
    test_r2, y_true, y_pred = evaluate(model, test_loader)
    print(f"   🏆 Final Test R2: {test_r2:.4f}")

    return {'name': name, 'r2': test_r2, 'loss': loss_hist, 'val_r2': r2_hist, 'pred': (y_true, y_pred)}

# ==============================================================================
# 5. Main Execution
# ==============================================================================
def main():
    # 1. Load Data
    csv_path = 'SweetpredDB.csv'
    if not os.path.exists(csv_path):
        print("Creating dummy data...")
        df = pd.DataFrame({'Smiles': ['C']*50 + ['CC']*50, 'logSw': np.random.rand(100)})
    else:
        df = pd.read_csv(csv_path)
        df.rename(columns=str.lower, inplace=True)
        df.rename(columns={'smiles': 'Smiles', 'logsw': 'logSw'}, inplace=True)

    tokenizer = SMILESTokenizer()
    tokenizer.build_vocab(df['Smiles'].tolist())
    pad_id = tokenizer.vocab["<pad>"]

    # 保存 Tokenizer (因为预测时需要)
    with open(os.path.join(SAVE_DIR, 'vocab.json'), 'w') as f:
        json.dump(tokenizer.vocab, f)

    # 强制删除旧缓存，确保特征是最新的
    dataset_cache = os.path.join(SAVE_DIR, "universal_data_v3.pt")
    if os.path.exists(dataset_cache): os.remove(dataset_cache)

    dataset = UniversalDataset(df, tokenizer, save_path=dataset_cache)

    # 2. Split with Indices (Critical for Analysis!)
    indices = np.arange(len(dataset))
    train_idx, temp_idx = train_test_split(indices, test_size=0.2, random_state=42)
    val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42)

    # Save Indices
    split_info = {'train': train_idx.tolist(), 'val': val_idx.tolist(), 'test': test_idx.tolist()}
    with open(os.path.join(SAVE_DIR, 'split_indices.json'), 'w') as f:
        json.dump(split_info, f)

    # Save Test Metadata (SMILES and True Labels) for Deep Analysis
    test_meta = []
    for idx in test_idx:
        test_meta.append({
            'index': int(idx),
            'SMILES': dataset.raw_smiles[idx],
            'True_LogSw': dataset.raw_labels[idx]
        })
    pd.DataFrame(test_meta).to_csv(os.path.join(SAVE_DIR, "test_metadata.csv"), index=False)
    print("📝 Test metadata saved.")

    # Create Subsets and Loaders
    train_ds = Subset(dataset, train_idx)
    val_ds = Subset(dataset, val_idx)
    test_ds = Subset(dataset, test_idx)

    loaders = (
        DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=universal_collate),
        DataLoader(val_ds, batch_size=32, shuffle=False, collate_fn=universal_collate),
        DataLoader(test_ds, batch_size=32, shuffle=False, collate_fn=universal_collate)
    )

    # 3. Experiments
    experiments = [
        ("Graph Only", {'use_graph': True, 'use_seq': False, 'use_expert': False, 'use_bert': False}),
        ("SMILES Only", {'use_graph': False, 'use_seq': True, 'use_expert': False, 'use_bert': False}),
        ("G+S+Exp", {'use_graph': True, 'use_seq': True, 'use_expert': True, 'use_bert': False}),
        ("G+S+BERT", {'use_graph': True, 'use_seq': True, 'use_expert': False, 'use_bert': True}),
        ("All-In-One", {'use_graph': True, 'use_seq': True, 'use_expert': True, 'use_bert': True})
    ]

    # 4. Run & Save
    all_results = []
    for name, cfg in experiments:
        res = run_experiment(name, cfg, len(tokenizer.vocab), pad_id, loaders, epochs=100)
        all_results.append(res)

    # Save Experiment Results Pickle
    # Convert numpy arrays to lists for JSON serialization if needed, or use pickle
    with open(os.path.join(SAVE_DIR, "all_results.pkl"), "wb") as f:
        pickle.dump(all_results, f)
    print(f"💾 All results saved to {SAVE_DIR}/all_results.pkl")

    # 5. Visualization (Basic)
    print("\n" + "="*40)
    print("📊 COMPARISON SUMMARY")
    print("="*40)

    fig = plt.figure(figsize=(18, 6))

    # Bar Chart
    ax1 = plt.subplot(1, 3, 1)
    names = [r['name'] for r in all_results]
    scores = [r['r2'] for r in all_results]
    bars = ax1.bar(names, scores, color=sns.color_palette("viridis", len(names)))
    ax1.set_title("Test R2 Score Comparison")
    ax1.set_ylim(0, 1.0)
    ax1.set_xticks(range(len(names)))
    ax1.set_xticklabels(names, rotation=45, ha="right")
    for bar in bars:
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f"{bar.get_height():.3f}", ha='center', va='bottom')

    # Val R2 Curves
    ax2 = plt.subplot(1, 3, 2)
    for res in all_results:
        ax2.plot(res['val_r2'], label=res['name'])
    ax2.set_title("Validation R2 History")
    ax2.legend()

    # Scatter (Best Model)
    ax3 = plt.subplot(1, 3, 3)
    best_res = max(all_results, key=lambda x: x['r2'])
    y_true, y_pred = best_res['pred']
    ax3.scatter(y_true, y_pred, alpha=0.5, color='#2ca02c', label=f"Best: {best_res['name']}")
    ax3.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'k--')
    ax3.set_title(f"Prediction: {best_res['name']}")
    ax3.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, "summary_plot.png"))
    plt.show()

if __name__ == '__main__':
    main()

## **结果分析**

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
from tqdm import tqdm

# ==============================================================================
# 0. 配置与加载
# ==============================================================================
SAVE_DIR = "experiment_results"
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 150

# 预定义的颜色映射（确保名字与训练时的 Name 一致）
# 如果你的模型名字不一样，请在这里修改 Key
MODEL_COLORS = {
    'Graph Only': '#999999',      # 灰
    'SMILES Only': '#e6550d',     # 橙
    'G+S+Exp': '#3182bd',         # 蓝
    'G+S+BERT': '#756bb1',        # 紫
    'All-In-One': '#2ca02c'       # 绿
}

def load_and_prep():
    print("📥 正在加载数据...")
    try:
        with open(os.path.join(SAVE_DIR, "all_results.pkl"), "rb") as f:
            all_results = pickle.load(f)
        df = pd.read_csv(os.path.join(SAVE_DIR, "test_metadata.csv"))
    except:
        print("❌ 数据缺失，请先运行训练代码！")
        return None, None, None

    # 计算性质
    print("⚗️ 计算分子性质...")
    mols = [Chem.MolFromSmiles(s) for s in df['SMILES']]
    df['MolWt'] = [Descriptors.MolWt(m) if m else 0 for m in mols]
    df['LogP'] = [Descriptors.MolLogP(m) if m else 0 for m in mols]
    df['TPSA'] = [Descriptors.TPSA(m) if m else 0 for m in mols]

    model_names = []
    for res in all_results:
        name = res['name']
        model_names.append(name)
        preds = res['pred'][1]
        if len(preds) == len(df):
            df[f'AbsErr_{name}'] = np.abs(df['True_LogSw'] - preds)

    # 检查颜色字典是否覆盖了所有模型
    # 如果没有，自动补充默认颜色，防止报错
    unique_models = model_names
    palette = []
    for name in unique_models:
        if name in MODEL_COLORS:
            palette.append(MODEL_COLORS[name])
        else:
            print(f"⚠️ 警告: 模型 '{name}' 没有预定义颜色，使用默认黑色。")
            palette.append('#000000')

    return df, model_names, mols, palette

# ==============================================================================
# 1. 修复后的属性切片分析 (Error Trend)
# ==============================================================================
def plot_property_slicing_fixed(df, model_names, palette):
    """
    绘制误差随 MW, LogP, TPSA 变化的趋势图。
    """
    print("📊 正在绘制属性趋势图...")

    props = [
        ('MolWt', 'Molecular Weight'),
        ('LogP', 'LogP (Polarity)'),
        ('TPSA', 'TPSA (Complexity)')
    ]

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    for i, (col, title) in enumerate(props):
        ax = axes[i]

        # 1. 数据准备：扁平化 (Melt)
        # 我们需要一个长格式 DataFrame: [Value, Error, Model]
        plot_data = []

        # 先对属性分桶 (5等分)
        df[f'{col}_Bin'] = pd.qcut(df[col], q=5, duplicates='drop')

        for name in model_names:
            # 计算每个桶的平均误差
            # group: (Bin_Interval, Mean_Error)
            group = df.groupby(f'{col}_Bin')[f'AbsErr_{name}'].mean().reset_index()
            group.columns = ['Bin', 'MAE']
            group['Model'] = name
            # 取区间中点作为 X 轴数值 (方便绘图)
            group['X_Value'] = group['Bin'].apply(lambda x: x.mid)
            plot_data.append(group)

        final_df = pd.concat(plot_data)

        # 2. 绘图
        # 注意：这里直接传入 palette list，顺序必须和 hue_order 一致
        sns.lineplot(
            data=final_df,
            x='X_Value',
            y='MAE',
            hue='Model',
            hue_order=model_names,
            palette=palette,
            marker='o',
            linewidth=2.5,
            ax=ax
        )

        ax.set_title(f"Error Trend by {title}", fontweight='bold')
        ax.set_ylabel("Mean Absolute Error (MAE)")
        ax.set_xlabel(title)
        if i == 0:
            ax.legend(title='Model')
        else:
            ax.legend().remove() # 只在第一张图显示图例，避免遮挡

    plt.tight_layout()
    plt.show()

# ==============================================================================
# 2. 分模型化学空间误差图 (Separate Maps)
# ==============================================================================
def plot_separate_error_maps(df, model_names, mols):
    print("🗺️ 正在绘制 5 张独立的化学空间误差图...")

    # 1. 计算 t-SNE (共用坐标)
    valid_indices = [i for i, m in enumerate(mols) if m]
    if not valid_indices: return

    fps = [AllChem.GetMorganFingerprintAsBitVect(mols[i], 2, nBits=1024) for i in valid_indices]
    tsne = TSNE(n_components=2, random_state=42, init='pca', learning_rate='auto')
    X_emb = tsne.fit_transform(np.array(fps))

    plot_df = df.iloc[valid_indices].copy()
    plot_df['x'] = X_emb[:, 0]
    plot_df['y'] = X_emb[:, 1]

    # 2. 统一色阶 (关键！)
    # 找出所有模型中的最大误差值 (95% 分位)，以此为上限，保证颜色可比性
    all_err_vals = []
    for name in model_names:
        all_err_vals.extend(plot_df[f'AbsErr_{name}'].values)
    vmax = np.percentile(all_err_vals, 95)

    # 3. 循环绘图 (每个模型一张图)
    # 布局：2行3列
    fig, axes = plt.subplots(2, 3, figsize=(18, 11))
    axes = axes.flatten()

    for i, name in enumerate(model_names):
        ax = axes[i]

        # 获取误差
        errors = plot_df[f'AbsErr_{name}']

        # 散点图
        # viridis_r: 紫色=低误差(好)，黄色=高误差(差)
        sc = ax.scatter(
            plot_df['x'], plot_df['y'],
            c=errors,
            cmap='viridis_r',
            vmin=0, vmax=vmax,
            s=30, alpha=0.8, edgecolors='k', linewidth=0.1
        )

        ax.set_title(f"{name}", fontsize=14, fontweight='bold', color=MODEL_COLORS.get(name, 'black'))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.grid(False) # 去掉网格更干净

    # 隐藏多余子图 (如果有)
    for j in range(len(model_names), len(axes)):
        axes[j].axis('off')

    # 添加统一 Colorbar
    cbar_ax = fig.add_axes([0.92, 0.2, 0.02, 0.6])
    cbar = fig.colorbar(sc, cax=cbar_ax)
    cbar.set_label('Absolute Prediction Error (MAE)', fontsize=12)

    plt.suptitle("Chemical Space Error Maps (Separate Models)\n(Darker/Purple = Better, Brighter/Yellow = Worse)", fontsize=18, y=0.96)
    plt.show()

# ==============================================================================
# 3. 主执行逻辑
# ==============================================================================
def main():
    # 1. 加载
    df, model_names, mols, palette = load_and_prep()
    if df is None: return

    print(f"📊 待分析模型: {model_names}")

    # 2. 绘制属性趋势图 (修复版)
    plot_property_slicing_fixed(df, model_names, palette)

    # 3. 绘制分模型化学空间图
    plot_separate_error_maps(df, model_names, mols)

if __name__ == "__main__":
    main()